In [10]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

from src.lab08.langchain_tools import tools, tool_dict
from src.lab08.langchain_tools import get_current_time, get_yf_stock_history

In [2]:
load_dotenv()   # .env 파일의 api_key 정보를 환경 변수로 로딩.

True

# 한 번에 출력하기

## 언어 모델만 있는 경우

In [3]:
model = ChatOpenAI(model='gpt-4o-mini')

In [4]:
parser = StrOutputParser()

In [5]:
chain = model | parser

In [6]:
messages = [HumanMessage('한강 작가의 채식주의자를 1000자 이내로 요약해줘.')]

In [7]:
ai_message = chain.invoke(input=messages)

In [8]:
print(ai_message)

한강의 소설 "채식주의자"는 3부로 구성된 이야기로, 주인공 영혜의 비극적인 변화를 중심으로 전개됩니다. 이야기의 시작은 평범한 주부인 영혜가 갑작스럽게 채식주의자가 되겠다고 선언하면서 시작됩니다. 그녀는 고기를 일체 먹지 않겠다고 결심하고, 이를 통해 자신이 처한 가족과 사회의 억압적인 환경에서 벗어나고자 합니다.

영혜의 변화를 이해하려는 가족은 혼란에 빠지고, 그녀의 남편인 상욱은 영혜의 결정이 자신과의 관계에 미치는 영향을 받아 괴로워합니다. 영혜의 가족은 그녀의 선택을 이해하기는커녕, 점차 그녀를 소외시키고 비난하게 됩니다. 영혜의 채식주의는 단순한 식습관의 변화가 아니라, 내면의 고통과 개인의 자유에 대한 갈망을 상징합니다.

소설은 영혜가 갇힌 현실에서 탈출하려는 시도로 해석될 수 있으며, 그 과정에서 인간의 본성과 욕망, 그리고 사회적 압박에 대한 깊은 성찰을 제공합니다. 글은 현대 사회에서의 인간관계, 자아의 정체성 및 개인의 선택이 어떻게 달라질 수 있는지를 탐구하며, 마지막에는 영혜의 삶이 어떻게 비극적으로 마무리되는지를 보여줍니다. "채식주의자"는 결국 인간의 내면과 사회적 갈등에 대한 날카로운 고찰을 담고 있습니다.


## 도구를 설정한 경우 한 번에 출력하기

In [11]:
# AI 모델과 도구 목록을 바인딩.
model_with_tools = model.bind_tools(tools=tools)

In [12]:
messages = [HumanMessage('서울, 부산, LA의 현재 시간을 비교해줘.')]

In [13]:
ai_message = model_with_tools.invoke(input=messages)

In [14]:
ai_message.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_aenXL72Wu2vHSsSqqMi6voyw)
 Call ID: call_aenXL72Wu2vHSsSqqMi6voyw
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_rDtDmMg0Puf4f0U9BeTr1gfJ)
 Call ID: call_rDtDmMg0Puf4f0U9BeTr1gfJ
  Args:
    timezone: Asia/Seoul
    location: 부산
  get_current_time (call_cbPPD8uwUckEwVppYcgRX1p8)
 Call ID: call_cbPPD8uwUckEwVppYcgRX1p8
  Args:
    timezone: America/Los_Angeles
    location: LA


In [15]:
messages.append(ai_message)     # tool_calls를 가지고 있는 AIMessages를 대화 리스트에 추가!

In [21]:
for tool_call in ai_message.tool_calls:
    print(tool_call['name'], tool_call['args'])
    fn = tool_dict[tool_call['name']]       # tool_dict.get(tool_call['name'])
    tool_msg = fn.invoke(input=tool_call)
    messages.append(tool_msg)

get_current_time {'timezone': 'Asia/Seoul', 'location': '서울'}
get_current_time {'timezone': 'Asia/Seoul', 'location': '부산'}
get_current_time {'timezone': 'America/Los_Angeles', 'location': 'LA'}


In [22]:
for m in messages:
    m.pretty_print()

================================ Human Message =================================

서울, 부산, LA의 현재 시간을 비교해줘.
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_aenXL72Wu2vHSsSqqMi6voyw)
 Call ID: call_aenXL72Wu2vHSsSqqMi6voyw
  Args:
    timezone: Asia/Seoul
    location: 서울
  get_current_time (call_rDtDmMg0Puf4f0U9BeTr1gfJ)
 Call ID: call_rDtDmMg0Puf4f0U9BeTr1gfJ
  Args:
    timezone: Asia/Seoul
    location: 부산
  get_current_time (call_cbPPD8uwUckEwVppYcgRX1p8)
 Call ID: call_cbPPD8uwUckEwVppYcgRX1p8
  Args:
    timezone: America/Los_Angeles
    location: LA
================================= Tool Message =================================
Name: get_current_time

2025-09-16 15:00:38 Asia/Seoul(서울)
================================= Tool Message =================================
Name: get_current_time

2025-09-16 15:00:38 Asia/Seoul(부산)
================================= Tool Message =================================
Name: g

In [23]:
ai_message = model_with_tools.invoke(input=messages)

In [24]:
ai_message.pretty_print()

================================== Ai Message ==================================

현재 시간은 다음과 같습니다:

- 서울: 2025-09-16 15:00:38
- 부산: 2025-09-16 15:00:38
- LA: 2025-09-15 23:00:38

서울과 부산은 동일한 시간이지만, LA는 한국보다 16시간 늦은 시간을 보여줍니다.


# 언어 모델만 있는 경우 스트리밍 방식으로 출력하기

In [37]:
messages = [HumanMessage('한강 작가의 채식주의자를 1000자 이내로 요약해줘.')]

In [38]:
response = model.stream(input=messages)

In [39]:
type(response)  #> generator: for-in 구문에서 반복할 때마다 어떤 값을 yield(반환)하는 객체.

generator

In [40]:
for r in response:
    print(r.content, end='')

한강의 소설 *채식주의자*는 주인공 영혜가 갑작스럽게 채식주의자가 되면서 시작되는 이야기를 그립니다. 영혜는 평범한 가정주부로 살고 있었지만, 어느 날 꿈 속에서의 경험을 통해 육식을 거부하게 되며 삶의 방식을 급격히 변화시킵니다. 이 결정은 그녀의 가족과 주변 사람들에게 큰 충격을 주고, 갈등을 일으키게 됩니다.

소설은 세 개의 파트로 나뉘어 있고, 각각의 파트는 영혜의 남편, 형부, 그리고 동생의 시각에서 바라본 이야기를 담고 있습니다. 첫 번째 파트에서는 남편의 시점에서 영혜의 변화를 어떻게 받아들이는지를 보여주며, 그녀의 신념과 고뇌를 탐구합니다. 두 번째 파트에서는 형부가 영혜에게 집착하면서 발생하는 복잡한 감정선과 갈등을 다룹니다. 마지막으로, 세 번째 파트에서는 영혜의 동생이 그녀의 삶을 돌아보며 가족 간의 관계와 소외감을 탐구합니다.

*채식주의자*는 인간 존재와 선택, 그리고 사회의 규범에 대한 질문을 던지며, 개인의 자유와 고뇌를 심오하게 탐구합니다. 영혜의 채식 선택은 단순한 식습관의 변화가 아니라, 더 깊은 상징성과 메시지를 지니고 있습니다. 이 작품은 현대 사회에서 인간의 정체성과 관계, 그리고 욕망을 성찰하는 중요한 이야기로 평가받습니다.

In [42]:
# model과 parser가 연결된 체인을 사용하는 경우
response = chain.stream(input=[HumanMessage('해리포터와 비밀의 방의 내용을 1000자 이내로 요약해줘.')])
for r in response:
    print(r, end='')

《해리 포터와 비밀의 방》은 J.K. 롤링의 해리 포터 시리즈 두 번째 책으로, 해리 포터가 호그와트 마법학교의 2학년을 보내는 이야기입니다.

해리는 여름 방학을 모르고 지내고, 다시 호그와트에 돌아가기 위해 론 위즐리의 차를 타고 학교로 향합니다. 학교에 도착한 후, 해리는 이상한 사건들이 발생하고 있다는 소문을 듣게 됩니다. 여러 학생들이 의식 불명 상태로 쓰러지고, 교내에는 '비밀의 방'이라는 전설이 떠도는 상황입니다. 이 방에는 차별적인 존재가 숨겨져 있다고 합니다.

해리는 친구 론과 헤르미온느와 함께 비밀의 방의 진실을 밝히기 위해 조사에 나섭니다. 그 과정에서 해리는 자신의 능력과 정체성에 대해 더 깊이 이해하게 됩니다. 해리는 파셀탑스(뱀과 대화할 수 있는 능력)를 가진 것을 통해 비밀의 방의 존재와 괴물의 정체를 추적합니다.

결국 해리와 친구들은 비밀의 방을 찾고, 그곳에서 '바실리스크'라는 큰 뱀과 맞닥뜨리게 됩니다. 해리는 친구들이 도와줌으로써 바실리스크를 물리치고, 위험에 처한 지니 위즐리를 구합니다. 이 과정에서 해리는 자신의 용기와 우정을 다시 한번 확인하며, 자신의 정체성에 대한 중요한 단서를 얻습니다.

마침내, 해리는 덤블도어와의 대화를 통해 자신의 성장과 앞으로의 길에 대해 더 깊이 고민하게 되며, 이야기는 해리와 친구들이 함께한 모험의 의미를 보여줍니다. 이야기는 우정, 용기, 그리고 자기 발견의 중요성을 강조하며 종료됩니다.

## 도구를 포함하는 모델의 스트리밍 출력

In [43]:
messages = [HumanMessage('하이닉스의 지난 한 달 동안 주식 변동을 요약해줘.')]

In [44]:
response = model_with_tools.stream(input=messages)

In [45]:
# 파편하된 tool_calls 청크들을 하나로 합치기.
is_first = True
for chunk in response:
    print(type(chunk))  #> AIMessageChunk
    if is_first:    # 첫번째 청크이면,
        is_first = False    # 다음 반복(iteration)부터는 첫번째 청크가 아니기 때문에
        gathered = chunk
    else:   # 두번째 이후 청크이면,
        gathered += chunk   # 이미 만들어진 청크의 뒤에 덧붙임(append).
    gathered.pretty_print()

<class 'langchain_core.messages.ai.AIMessageChunk'>
============================ Aimessagechunk Message ============================
Tool Calls:
  get_yf_stock_history (call_EpspKkzlvTsjooYlrUvLm3uu)
 Call ID: call_EpspKkzlvTsjooYlrUvLm3uu
  Args:
<class 'langchain_core.messages.ai.AIMessageChunk'>
============================ Aimessagechunk Message ============================
Tool Calls:
  get_yf_stock_history (call_EpspKkzlvTsjooYlrUvLm3uu)
 Call ID: call_EpspKkzlvTsjooYlrUvLm3uu
  Args:
<class 'langchain_core.messages.ai.AIMessageChunk'>
============================ Aimessagechunk Message ============================
Tool Calls:
  get_yf_stock_history (call_EpspKkzlvTsjooYlrUvLm3uu)
 Call ID: call_EpspKkzlvTsjooYlrUvLm3uu
  Args:
<class 'langchain_core.messages.ai.AIMessageChunk'>
============================ Aimessagechunk Message ============================
Tool Calls:
  get_yf_stock_history (call_EpspKkzlvTsjooYlrUvLm3uu)
 Call ID: call_EpspKkzlvTsjooYlrUvLm3uu
  Args:
    inpu

In [47]:
messages.append(gathered)   # 하나로 합친 AI 답변을 메시지 리스트에 추가!

In [48]:
# 하나로 합쳐진 tool_calls를 반복하면서 function calling을 수행.
for tool_call in gathered.tool_calls:
    fn = tool_dict.get(tool_call['name'])
    tool_msg = fn.invoke(input=tool_call)
    messages.append(tool_msg)

In [55]:
for m in messages:
    m.pretty_print()

================================ Human Message =================================

하이닉스의 지난 한 달 동안 주식 변동을 요약해줘.
============================ Aimessagechunk Message ============================
Tool Calls:
  get_yf_stock_history (call_EpspKkzlvTsjooYlrUvLm3uu)
 Call ID: call_EpspKkzlvTsjooYlrUvLm3uu
  Args:
    input: {'ticker': '000660.KS', 'period': '1mo'}
============================ Aimessagechunk Message ============================
Tool Calls:
  get_yf_stock_history (call_EpspKkzlvTsjooYlrUvLm3uu)
 Call ID: call_EpspKkzlvTsjooYlrUvLm3uu
  Args:
    input: {'ticker': '000660.KS', 'period': '1mo'}
================================= Tool Message =================================
Name: get_yf_stock_history

| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |
|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|
| 2025-08-18 00:00:00+09:00 | 271608 | 271608 | 266615 |  26

In [56]:
response = model_with_tools.stream(input=messages)

In [57]:
for r in response:
    print(r.content, end='')

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_EpspKkzlvTsjooYlrUvLm3uu", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}